# Welcome to Exkaldi

In this section, we will further process the Kaldi decoding lattice and score the results.

In [ ]:
import exkaldi

import os
dataDir = "librispeech_dummy"

Load the lattice file (generated in 09_decode_back_HMM-GMM_and_WFST).

In [ ]:
latFile = os.path.join(dataDir, "exp", "decode_test", "test.lat")

lat = exkaldi.decode.wfst.load_lat(latFile)

lat

To be simple and straightforward, we get the 1-best result from lattice. Word-id table and HMM model are necessary.

Word-id table can be words.txt file (If decoded in word level) or phones.txt file (If decoded in phone level) or Exkaldi ListTable object. 
Ideally, LexiconBank object is also avaliable because you can get both "words" and "phones" from it.

In [ ]:
wordsFile = os.path.join(dataDir, "exp", "words.txt")

hmmFile = os.path.join(dataDir, "exp", "train_delta", "final.mdl")

In [ ]:
result = lat.get_1best(wordSymbolTable=wordsFile, hmm=hmmFile, lmwt=1, acwt=0.5)

result.subset(nHead=1)

___result___ is a exkaldi __Transcription__ object (a subclass of Python dict).

The decoding result is int-ID format. If you want it by text-format, try this:

In [ ]:
textResult = exkaldi.hmm.transcription_from_int(result, wordsFile)

textResult.subset(nHead=1)

Just for convenience, we restorage lexicons.

In [ ]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.decode.graph.load_lex(lexFile)

lexicons

In [ ]:
del textResult

Besides the __transcription_from_int__ function, we can transform transcription by using the __Transcription__'s own method, like this:

In [ ]:
word2id = lexicons("words")
oovID = word2id[lexicons("oov")]
id2word = word2id.reverse()

textResult = result.convert(id2word, oovID)

textResult.subset(nHead=1)

In [ ]:
del result

Now we can score the decoding result. Typically, you can compute the WER(word err rate).

In [ ]:
refFile = os.path.join(dataDir, "test", "text")

score = exkaldi.decode.score.wer(ref=refFile, hyp=textResult, mode="present")

score

Or some times, compute the edit distance score. 

In [ ]:
score = exkaldi.decode.score.edit_distance(ref=refFile, hyp=textResult, mode="present")

score

Then compute the accuracy of words levels.

In [ ]:
1 - score.editDistance/score.words

We tested this and only get the WER 168.74, and the accuracy rate of words is 16.67%.

We support further process the lattice, for example, to add penalty or to scale it.

Here is a example to config different language model weight(LMWT) and penalty. (In Instead of text-format result, we use int-format reference file.)

In [ ]:
refInt = exkaldi.hmm.transcription_to_int(refFile, lexicons("words"), unkSymbol=lexicons("oov"))
refIntFile = os.path.join(dataDir, "exp", "decode_test", "text.int")
refInt.save(refIntFile)

refInt.subset(nHead=1)

In [ ]:
for penalty in [0., 0.5, 1.0]:
    for LMWT in range(10, 15):
        
        newLat = lat.add_penalty(penalty)
        result = newLat.get_1best(lexicons("words"), hmmFile, lmwt=LMWT, acwt=0.5)

        score = exkaldi.decode.score.wer(ref=refInt, hyp=result, mode="present")
        
        print(f"Penalty {penalty}, LMWT {LMWT}: WER {score.WER}")

From the lattice, you can get the phone-level result.

In [ ]:
phoneResult = lat.get_1best(lexicons("phones"), hmmFile, lmwt=1, acwt=0.5, phoneLevel=True)

phoneResult = exkaldi.hmm.transcription_from_int(phoneResult, lexicons("phones"))

phoneResult.subset(nHead=1)

From lattice, N-Best results can also be extracted.

In [ ]:
result = lat.get_nbest(
                        n=3,
                        wordSymbolTable=lexicons("words"),
                        hmm=hmmFile, 
                        acwt=0.5, 
                        phoneLevel=False,
                        requireCost=False,
                )

for re in result:
    print(re.name, type(re))

___phoneResult___ is a list of N-bests __Transcription__ objects. If ___requireCost___ is True, return the LM score and AM score sumultaneously.

In [ ]:
result = lat.get_nbest(
                        n=3,
                        wordSymbolTable=lexicons("words"),
                        hmm=hmmFile, 
                        acwt=0.5, 
                        phoneLevel=False,
                        requireCost=True,
                )

for re in result[0]:
    print(re.name, type(re))
    
for re in result[1]:
    print(re.name, type(re))

for re in result[2]:
    print(re.name, type(re))

And importantly, Alignment can be returned to support discriminative training. 

In [ ]:
result = lat.get_nbest(
                        n=3,
                        wordSymbolTable=lexicons("words"),
                        hmm=hmmFile, 
                        acwt=0.5, 
                        phoneLevel=False,
                        requireCost=False,
                        requireAli=True,
                )

for re in result[1]:
    print(re.name, type(re))